In [8]:
#### ORTALAMA YERİNE MEDİAN DEĞERİ SİYAH KESİKLİ ÇİZGİ İLE GÖSTERİLİYOR ####

#### Te, Ne ve element bollukları log alınmış değerleri ile histogram oluşturarak tek bir pdf dosyasında birleştiriyor. ####

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages

# Excel dosyasını oku
excel_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/Disc_Region.xlsx"
neat_result_df = pd.read_excel(excel_yolu, sheet_name='neat_result')
renklendirilmis_df = pd.read_excel(excel_yolu, sheet_name='Sayfa2')

# Histogramlar için klasör oluştur
klasor_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/ThickDisc/"
if not os.path.exists(klasor_yolu):
    os.makedirs(klasor_yolu)

# Ortalamalar için dosya oluştur
ortalama_dosyasi_yolu = os.path.join(klasor_yolu, "ortalama_degerler.txt")

# Renklendirme fonksiyonu
def get_color(component):
    if component == 'Bulge':
        return 'blue'
    elif component == 'Thin Disc':
        return 'red'
    elif component == 'Thick Disc':
        return 'green'
    elif component == 'Halo':
        return 'orange'
    else:
        return None

# Renklendirme bilgilerini ekle
renklendirilmis_df['color'] = renklendirilmis_df['Gbileşen'].apply(get_color)
renk_dict = renklendirilmis_df.set_index('idPNMain')['color'].to_dict()

# Sütun başlıkları
sutun_basliklari = ['c(Hb) (Ha/Hb)', 'c(Hb) (Hg/Hb)', 'c(Hb) (Hd/Hb)', '[OII] density', '[S II] Density', '[ArIV] density', 
                    '[OII] temperature', '[S II] temperature', '[N II] temperature', 
                    '[OI] temperature', '[O III] Temperature', '[NeIII] temperature', 
                    '[ArIII] temperature', '[ArV] temperature', 'He I temperature (7281/6678)', 
                    'C/H', 'N/H', 'O/H', 'Ne/H', 'Ar/H', 'S/H', 'Cl/H', 'He/H', 'Diameter', 'Angular Width']

# Histogram parametreleri
histogram_parametreleri = {
    'c(Hb) (Ha/Hb)': {'bins': 50, 'range': (0, 6)},
    'c(Hb) (Hg/Hb)': {'bins': 50, 'range': (0, 6)},
    'c(Hb) (Hd/Hb)': {'bins': 50, 'range': (0, 6)},
    '[OII] density': {'bins': 50, 'range': (0, 20000)},
    '[S II] Density': {'bins': 50, 'range': (0, 15000)},
    '[ArIV] density': {'bins': 50, 'range': (0, 70000)},
    '[OII] temperature': {'bins': 50, 'range': (5000, 35000)},
    '[S II] temperature': {'bins': 50, 'range': (0, 35000)},
    '[N II] temperature': {'bins': 50, 'range': (5000, 35000)},
    '[OI] temperature': {'bins': 50, 'range': (0, 35000)},
    '[O III] Temperature': {'bins': 50, 'range': (5000, 35000)},
    '[NeIII] temperature': {'bins': 50, 'range': (0, 35000)},
    '[ArIII] temperature': {'bins': 50, 'range': (5000, 35000)},
    '[ArV] temperature': {'bins': 50, 'range': (0, 35000)},
    'He I temperature (7281/6678)': {'bins': 50, 'range': (0, 4e5)},
    'C/H': {'bins': 50, 'range': (0, 1e-3)},
    'N/H': {'bins': 50, 'range': (0, 1e-3)},
    'O/H': {'bins': 50, 'range': (0, 3e-3)},
    'Ne/H': {'bins': 50, 'range': (0, 3e-3)},
    'Ar/H': {'bins': 50, 'range': (0, 175e-7)},
    'S/H': {'bins': 50, 'range': (0, 3e-5)},
    'Cl/H': {'bins': 50, 'range': (0, 0.6e-5)},
    'He/H': {'bins': 50, 'range': (0, 0.5)},
    'Diameter': {'bins': 50, 'range': (0, 200)},
    'Angular Width': {'bins': 50, 'range': (0, 4)}
}

# Renklere karşılık gelen etiketler
renk_etiketleri = {
    'red': 'Thin Disc',
    'green': 'Thick Disc',
    'blue': 'Bulge',
    'orange': 'Halo'
}

# Ortalamaları yazmak için dosyayı aç
with open(ortalama_dosyasi_yolu, 'w') as f:
    # Her sütun için histogram oluştur
    for baslik in sutun_basliklari:
        plt.figure()
        bins = histogram_parametreleri[baslik]['bins']
        range_ = histogram_parametreleri[baslik]['range']
        ortalamalar = {}
        
        # 0 değerleri hariç tutarak tüm verilerin medyanını hesapla ve siyah kesikli çizgi olarak göster
        tum_veriler_median = neat_result_df[baslik][neat_result_df[baslik] != 0].median()
        plt.axvline(tum_veriler_median, color='black', linestyle='dashed', linewidth=1)
        print(f"{baslik} - Tüm Veriler Medyanı (0 hariç): {tum_veriler_median:.2e}")
        f.write(f"{baslik} - Tüm Veriler Medyanı (0 hariç): {tum_veriler_median:.2e}\n")
        
        for color, etiket in renk_etiketleri.items():
            filtreli_veri = neat_result_df[neat_result_df['idPNMain'].map(renk_dict) == color][baslik]
            filtreli_veri = filtreli_veri[(filtreli_veri != 0) & (filtreli_veri != 35000)]
            
            if not filtreli_veri.empty:
                plt.hist(filtreli_veri, bins=bins, range=range_, edgecolor='black', color=color, label=etiket)
                medyan = filtreli_veri.median()
                ortalamalar[etiket] = medyan
        
        # Medyan değerleri kesikli çizgilerle göster
        for etiket, medyan in ortalamalar.items():
            color = [k for k, v in renk_etiketleri.items() if v == etiket][0]
            plt.axvline(medyan, color=color, linestyle='dashed', linewidth=1)
            print(f"{baslik} - {etiket} Medyan (0 hariç): {medyan:.2e}")
            f.write(f"{baslik} - {etiket} Medyan (0 hariç): {medyan:.2e}\n")
        
        # Plot ayarları
        if baslik in ['c(Hb) (Ha/Hb)', 'c(Hb) (Hg/Hb)', 'c(Hb) (Hd/Hb)']:
            plt.xlabel("cH(β)")
        else:
            plt.xlabel(baslik)
        
        plt.ylabel("Number")
        #plt.legend(fontsize=12)
        
        # Histogramı kaydet
        kaydet_yolu = os.path.join(klasor_yolu, f"{baslik.replace('/', '_')}.png")
        plt.savefig(kaydet_yolu, dpi=300)
        
        # Plot'u temizle
        plt.clf()

print("Histogramlar başarıyla oluşturuldu ve medyanlar listeye yazıldı.")

# PDF dosyası yolu
pdf_dosyasi_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/ThickDisc/selected_histograms.pdf"

# Histogram başlıkları
basliklar = ['[S II] Density', '[N II] temperature', '[O III] Temperature']

# PDF ile grafik oluşturma
with PdfPages(pdf_dosyasi_yolu) as pdf:
    plt.figure(figsize=(12, 12))  # Şekil boyutunu ayarlayın

    for i, baslik in enumerate(basliklar):
        plt.subplot(len(basliklar), 1, i + 1)  # 1 satır, len(basliklar) sütunlu alt grafikleri oluşturur
        bins = histogram_parametreleri[baslik]['bins']
        range_ = histogram_parametreleri[baslik]['range']
        ortalamalar = {}
        
        # 0 değerleri hariç tutarak tüm verilerin medyanını hesapla ve siyah kesikli çizgi olarak göster
        tum_veriler_median = neat_result_df[baslik][neat_result_df[baslik] != 0].median()
        plt.axvline(tum_veriler_median, color='black', linestyle='dashed', linewidth=1)
        
        for color, etiket in renk_etiketleri.items():
            filtreli_veri = neat_result_df[neat_result_df['idPNMain'].map(renk_dict) == color][baslik]
            filtreli_veri = filtreli_veri[(filtreli_veri != 0) & (filtreli_veri != 35000)]
            
            if not filtreli_veri.empty:
                plt.hist(filtreli_veri, bins=bins, range=range_, edgecolor='black', color=color, label=etiket)
                medyan = filtreli_veri.median()
                ortalamalar[etiket] = medyan
        
        for etiket, medyan in ortalamalar.items():
            color = [k for k, v in renk_etiketleri.items() if v == etiket][0]
            plt.axvline(medyan, color=color, linestyle='dashed', linewidth=1)
        
        plt.xlabel(baslik)
        plt.ylabel("Number")
        #plt.legend(fontsize=12)
        plt.tight_layout()  # Alt grafiklerin sıkışmaması için düzenler

    pdf.savefig()  # Tek bir sayfaya tüm alt grafiklerle birlikte kaydeder
    plt.clf()
    
# PDF dosyası yolu
pdf_dosyasi_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/ThickDisc/selected_histograms.pdf"

# Histogram başlıkları
basliklar = ['[S II] Density', '[N II] temperature', '[O III] Temperature']

# PDF ile grafik oluşturma
with PdfPages(pdf_dosyasi_yolu) as pdf:
    plt.figure(figsize=(12, 12))  # Şekil boyutunu ayarlayın

    for i, baslik in enumerate(basliklar):
        plt.subplot(len(basliklar), 1, i + 1)  # 1 satır, len(basliklar) sütunlu alt grafikleri oluşturur
        bins = histogram_parametreleri[baslik]['bins']
        range_ = histogram_parametreleri[baslik]['range']
        ortalamalar = {}
        
        # 0 değerleri hariç tutarak tüm verilerin medyanını hesapla ve siyah kesikli çizgi olarak göster
        tum_veriler_median = neat_result_df[baslik][neat_result_df[baslik] != 0].median()
        plt.axvline(tum_veriler_median, color='black', linestyle='dashed', linewidth=1)
        
        for color, etiket in renk_etiketleri.items():
            filtreli_veri = neat_result_df[neat_result_df['idPNMain'].map(renk_dict) == color][baslik]
            filtreli_veri = filtreli_veri[(filtreli_veri != 0) & (filtreli_veri != 35000)]
            
            if not filtreli_veri.empty:
                plt.hist(filtreli_veri, bins=bins, range=range_, edgecolor='black', color=color, label=etiket)
                medyan = filtreli_veri.median()
                ortalamalar[etiket] = medyan
        
        for etiket, medyan in ortalamalar.items():
            color = [k for k, v in renk_etiketleri.items() if v == etiket][0]
            plt.axvline(medyan, color=color, linestyle='dashed', linewidth=1)
        
        plt.xlabel(baslik)
        plt.ylabel("Number")
        #plt.legend(fontsize=12)
        plt.tight_layout()  # Alt grafiklerin sıkışmaması için düzenler

    pdf.savefig()  # Tek bir sayfaya tüm alt grafiklerle birlikte kaydeder
    plt.clf()

    
# PDF dosyası yolu
pdf_dosyasi_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/ThickDisc/size_histograms.pdf"

# Histogram başlıkları
basliklar = ['Diameter', 'Angular Width']

# PDF ile grafik oluşturma
with PdfPages(pdf_dosyasi_yolu) as pdf:
    plt.figure(figsize=(12, 6))  # Şekil boyutunu ayarlayın (genişlik, yükseklik)

    # Histogramlar için alt grafikler
    for i, baslik in enumerate(basliklar):
        plt.subplot(1, 2, i + 1)  # 2 satır, 2 sütunlu alt grafikleri oluşturur
        bins = histogram_parametreleri[baslik]['bins']
        range_ = histogram_parametreleri[baslik]['range']
        ortalamalar = {}
        
        # 0 değerleri hariç tutarak tüm verilerin medyanını hesapla ve siyah kesikli çizgi olarak göster
        tum_veriler_median = neat_result_df[baslik][neat_result_df[baslik] != 0].median()
        plt.axvline(tum_veriler_median, color='black', linestyle='dashed', linewidth=1)
        
        for color, etiket in renk_etiketleri.items():
            filtreli_veri = neat_result_df[neat_result_df['idPNMain'].map(renk_dict) == color][baslik]
            filtreli_veri = filtreli_veri[(filtreli_veri != 0) & (filtreli_veri != 35000)]
            
            if not filtreli_veri.empty:
                plt.hist(filtreli_veri, bins=bins, range=range_, edgecolor='black', color=color, label=etiket)
                medyan = filtreli_veri.median()
                ortalamalar[etiket] = medyan
        
        for etiket, medyan in ortalamalar.items():
            color = [k for k, v in renk_etiketleri.items() if v == etiket][0]
            plt.axvline(medyan, color=color, linestyle='dashed', linewidth=1)
        
        plt.xlabel(baslik)
        plt.ylabel("Number")
        #plt.legend(fontsize=12)
        
    pdf.savefig()  # Tek bir sayfaya tüm alt grafiklerle birlikte kaydeder
    plt.clf()

    
# Kimyasal bolluklar için ayrı bir PDF dosyası oluşturma
# PDF dosyası yolu
bolluk_pdf_dosyasi_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/ThickDisc/abundances_histograms.pdf"

# Histogram başlıkları
basliklar = ['He/H', 'N/H', 'O/H', 'Ne/H', 'S/H', 'Cl/H', 'Ar/H']

# PDF ile grafik oluşturma
with PdfPages(bolluk_pdf_dosyasi_yolu) as pdf:
    plt.figure(figsize=(12, 12))  # Şekil boyutunu ayarlayın

    for i, baslik in enumerate(basliklar):
        plt.subplot(4, 2, i + 1)  # 4 satır, 2 sütunlu alt grafikleri oluşturur
        bins = histogram_parametreleri[baslik]['bins']
        # range_ parametresini kaldırarak otomatik aralıkları kullanacağız
        ortalamalar = {}

        # Verilerin logaritmasını al (0 değerlerini hariç tut)
        log_veriler = np.log10(neat_result_df[baslik][neat_result_df[baslik] > 0])
        tum_veriler_median = log_veriler.median()
        plt.axvline(tum_veriler_median, color='black', linestyle='dashed', linewidth=1)

        for color, etiket in renk_etiketleri.items():
            filtreli_veri = neat_result_df[neat_result_df['idPNMain'].map(renk_dict) == color][baslik]
            filtreli_veri = filtreli_veri[(filtreli_veri != 0) & (filtreli_veri != 35000)]
            
            if not filtreli_veri.empty:
                log_filtreli_veri = np.log10(filtreli_veri)
                plt.hist(log_filtreli_veri, bins=bins, edgecolor='black', color=color, label=etiket)
                medyan = log_filtreli_veri.median()
                ortalamalar[etiket] = medyan
        
        for etiket, medyan in ortalamalar.items():
            color = [k for k, v in renk_etiketleri.items() if v == etiket][0]
            plt.axvline(medyan, color=color, linestyle='dashed', linewidth=1)
        
        plt.xlabel(f'log({baslik})')
        plt.ylabel("Number")
        #plt.legend(fontsize=12, loc='upper right')
        plt.tight_layout()  # Alt grafiklerin sıkışmaması için düzenler

    pdf.savefig()  # Tek bir sayfaya tüm alt grafiklerle birlikte kaydeder
    plt.clf()  # Şekli temizler
    
print("Belirtilen sütunlar için PDF dosyaları oluşturuldu.")

c(Hb) (Ha/Hb) - Tüm Veriler Medyanı (0 hariç): 1.34e+00
c(Hb) (Ha/Hb) - Thin Disc Medyan (0 hariç): 1.54e+00
c(Hb) (Ha/Hb) - Thick Disc Medyan (0 hariç): 9.25e-01
c(Hb) (Ha/Hb) - Bulge Medyan (0 hariç): 1.67e+00
c(Hb) (Ha/Hb) - Halo Medyan (0 hariç): 4.64e-01
c(Hb) (Hg/Hb) - Tüm Veriler Medyanı (0 hariç): 3.15e+00
c(Hb) (Hg/Hb) - Thin Disc Medyan (0 hariç): 3.02e+00
c(Hb) (Hg/Hb) - Thick Disc Medyan (0 hariç): 1.93e+00
c(Hb) (Hg/Hb) - Bulge Medyan (0 hariç): 3.58e+00
c(Hb) (Hg/Hb) - Halo Medyan (0 hariç): 2.46e+00
c(Hb) (Hd/Hb) - Tüm Veriler Medyanı (0 hariç): 1.50e+00
c(Hb) (Hd/Hb) - Thin Disc Medyan (0 hariç): 1.52e+00
c(Hb) (Hd/Hb) - Thick Disc Medyan (0 hariç): 1.17e+00
c(Hb) (Hd/Hb) - Bulge Medyan (0 hariç): 2.12e+00
c(Hb) (Hd/Hb) - Halo Medyan (0 hariç): 9.19e-01
[OII] density - Tüm Veriler Medyanı (0 hariç): 5.90e+02
[OII] density - Thin Disc Medyan (0 hariç): 5.40e+02
[OII] density - Thick Disc Medyan (0 hariç): 2.14e+02
[OII] density - Bulge Medyan (0 hariç): 2.13e+01
[OII] de

/var/folders/ry/6rcrw5vs4f33lx3wyt4f6m5w0000gn/T/ipykernel_93069/1798614509.py:89: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


S/H - Tüm Veriler Medyanı (0 hariç): 3.63e-06
S/H - Thin Disc Medyan (0 hariç): 3.89e-06
S/H - Thick Disc Medyan (0 hariç): 2.69e-06
S/H - Bulge Medyan (0 hariç): 4.17e-06
S/H - Halo Medyan (0 hariç): 1.14e-06
Cl/H - Tüm Veriler Medyanı (0 hariç): 5.74e-07
Cl/H - Thin Disc Medyan (0 hariç): 6.96e-07
Cl/H - Thick Disc Medyan (0 hariç): 4.56e-07
Cl/H - Bulge Medyan (0 hariç): 5.59e-07
Cl/H - Halo Medyan (0 hariç): 2.04e-07
He/H - Tüm Veriler Medyanı (0 hariç): 2.02e-01
He/H - Thin Disc Medyan (0 hariç): 2.18e-01
He/H - Thick Disc Medyan (0 hariç): 1.73e-01
He/H - Bulge Medyan (0 hariç): 2.18e-01
He/H - Halo Medyan (0 hariç): 1.21e-01
Diameter - Tüm Veriler Medyanı (0 hariç): 1.50e+01
Diameter - Thin Disc Medyan (0 hariç): 1.56e+01
Diameter - Thick Disc Medyan (0 hariç): 1.91e+01
Diameter - Bulge Medyan (0 hariç): 1.00e+01
Diameter - Halo Medyan (0 hariç): 1.58e+01
Angular Width - Tüm Veriler Medyanı (0 hariç): 3.36e-01
Angular Width - Thin Disc Medyan (0 hariç): 2.98e-01
Angular Width - 

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import os

# Excel dosyasını oku
excel_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/Disc_Region.xlsx"
df = pd.read_excel(excel_yolu, sheet_name='düşeyara')

# ERBIAS ve SPARM sütunlarını seç
erbias_sutunu = df['ERBIAS']
sparm_sutunu = df['SPARM']
gbilesen_sutunu = df['Gbileşen']

# '0' olan değerleri 'NaN' ile değiştir
erbias_sutunu.replace(0, np.nan, inplace=True)
sparm_sutunu.replace(0, np.nan, inplace=True)

# Kategori renklerini tanımla
renk_etiketleri = {
    'Bulge': 'blue',          # Bulge ise mavi
    'Thin Disc': 'red',       # Thin Disc ise kırmızı
    'Thick Disc': 'green',    # Thick Disk ise yeşil
    'Halo': 'orange',         # Halo ise turuncu
}

# Histogramın genel özellikleri
bar_width = 0.15  # Her bir bileşen için sütun genişliği

# ERBIAS histogramını oluşturacak fonksiyon
def plot_erbias_histogram(ax):
    categories = ['E', 'R', 'B', 'I', 'A', 'S', np.nan]
    category_names = ['E', 'R', 'B', 'I', 'A', 'S', 'NaN']
    x_positions = np.arange(len(categories))  # Her kategori için x eksenindeki pozisyonlar
    offset = np.zeros_like(x_positions, dtype=float)  # Her kategori için başlangıçta kaydırma 0

    for component, color in renk_etiketleri.items():
        counts = []

        for category in categories:
            filtered_gbilesen_sutunu = gbilesen_sutunu[erbias_sutunu == category]
            component_count = filtered_gbilesen_sutunu[filtered_gbilesen_sutunu == component].count()
            counts.append(component_count)

        # Sütunları çiz
        ax.bar(x_positions + offset, counts, bar_width, color=color, edgecolor='black', label=component)
        offset += bar_width  # Sonraki bileşen için kaydırmayı artır

    # X eksenini özelleştir
    ax.set_xticks(x_positions + bar_width * 1.5)
    ax.set_xticklabels(category_names)
    ax.set_xlabel('ERBIAS Categories')
    ax.set_ylabel('Number')
    #ax.legend(loc='upper right')

# SPARM histogramını oluşturacak fonksiyon
def plot_sparm_histogram(ax):
    categoriess = ['s', 'p', 'a', 'r', 'm', np.nan]
    category_namess = ['s', 'p', 'a', 'r', 'm', 'NaN']
    x_positions = np.arange(len(categoriess))  # Her kategori için x eksenindeki pozisyonlar
    offset = np.zeros_like(x_positions, dtype=float)  # Her kategori için başlangıçta kaydırma 0

    for component, color in renk_etiketleri.items():
        counts = []

        for category in categoriess:
            filtered_gbilesen_sutunu = gbilesen_sutunu[sparm_sutunu == category]
            component_count = filtered_gbilesen_sutunu[filtered_gbilesen_sutunu == component].count()
            counts.append(component_count)

        # Sütunları çiz
        ax.bar(x_positions + offset, counts, bar_width, color=color, edgecolor='black', label=component)
        offset += bar_width  # Sonraki bileşen için kaydırmayı artır

    # X eksenini özelleştir
    ax.set_xticks(x_positions + bar_width * 1.5)
    ax.set_xticklabels(category_namess)
    ax.set_xlabel('SPARM Categories')
    ax.set_ylabel('Number')
    #ax.legend(loc='upper right')

# Diğer histogramları oluşturacak fonksiyonlar
# Bileşenleri istenen sırada tanımla
renk_etiketleri = { 
    'Thin Disc': 'red',       # En son Thin Disc
    'Thick Disc': 'green',    # Daha sonra Thick Disk
    'Bulge': 'blue',          # Sonra Bulge
    'Halo': 'orange',         # Önce Halo
}

# Histogramları oluşturacak fonksiyonları güncelle
def plot_histogram(ax, veri_sutunu, gbilesen_sutunu, bins, title, xlabel, ylabel):
    veri_sutunu = pd.to_numeric(veri_sutunu, errors='coerce')
    filtered_df = df[veri_sutunu <= bins[-1]]
    filtered_veri_sutunu = filtered_df[veri_sutunu.name]
    filtered_gbilesen_sutunu = filtered_df[gbilesen_sutunu.name]

    hist_data = []
    ortanca_degerler = {}

    # Bileşenleri belirli bir sırayla döngüye al
    for component, color in renk_etiketleri.items():
        component_values = filtered_veri_sutunu[filtered_gbilesen_sutunu == component]
        counts, bins = np.histogram(component_values, bins=bins)
        hist_data.append((component, counts, bins, color))
        median_value = component_values.median()
        ortanca_degerler[component] = median_value

    # Histogramları çizim sırasına göre çiz
    for component, counts, bins, color in hist_data:
        ax.hist(bins[:-1], bins=bins, weights=counts, color=color, edgecolor='black', label=component)
        median_value = ortanca_degerler[component]
        ax.axvline(x=median_value, color=color, linestyle='--', label=f'{component} Median')

    # Genel ortancayı gösteren siyah bir çizgi ekle
    all_median_value = filtered_veri_sutunu.median()
    ax.axvline(x=all_median_value, color='black', linestyle='--', label='Overall Median')

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()

# PDF dosyasının yolunu belirle
pdf_dosyasi_yolu = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/HASH_Size_Morfo_median.pdf"

# PDF dosyası oluştur
with PdfPages(pdf_dosyasi_yolu) as pdf:
    fig, axs = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid
    axs = axs.flatten()

    # Histogramları oluştur
    plot_histogram(axs[0], df['MajDiam'], df['Gbileşen'], np.arange(0, 201, 2), 'Diameter Histogram', 'Diameter (′′)', 'Number')
    plot_histogram(axs[1], df['Açısal_genişlik'], df['Gbileşen'], np.arange(0, 4, 0.1), 'Angular Width Histogram', 'Angular Width (pc)', 'Number')
    plot_erbias_histogram(axs[2])
    plot_sparm_histogram(axs[3])

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

# Kontrol etmek için dosya yolu yazdır
if os.path.exists(pdf_dosyasi_yolu):
    print(f"PDF dosyası başarıyla kaydedildi: {pdf_dosyasi_yolu}")
else:
    print(f"PDF dosyası kaydedilemedi: {pdf_dosyasi_yolu}")


PDF dosyası başarıyla kaydedildi: /Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/8.34/histogramlar/HASH_Size_Morfo_median.pdf
